
{{title_its_nb3}}

{{intro}}

In the previous notebook, we worked through challenges that can be associated with working with larger-than-memory datasets. Here, we'll start where we left off, with a dataset that is organized in chronological order, and has a chunking strategy applied. 

In this notebook, we introduce vector data that describes spatial features that we're interested in. This notebook will:
- Look at how to parse and inspect important geographic metadata,   
- Project data to match the coordinate referese system (CRS) of another data object,  
- Join raster and vector data by clipping the raster data cube by the spatial extent of a vector data frame.  
- Write this clipped raster object to disk. 

{{break}}

::::{tab-set}
:::{tab-item} Outline

(content.Section_A)=
**[A. Read data using strategy identified in previous notebook](#a-read-data-using-strategy-identified-in-previous-notebook)**

(content.Section_B)=
**[B. Incorporate glacier outline (vector) data](#b-incorporate-glacier-outline-vector-data)**
- {{b1_its_nb3}}
- {{b2_its_nb3}}
- {{b3_its_nb3}}

(content.Section_C)=
**[C. Combine raster and vector data](#c-combine-raster-and-vector-data)**
- {{c1_its_nb3}}
- {{c2_its_nb3}}
:::
:::{tab-item} Learning Goals
{{concepts}}
- Understand how to parse and manage coordinate reference system metadata,  
- Visualize vector dataframes with static and interactive plots,  
- Spatial subsetting and clipping with vector and raster data,
- Write raster data to disk.


{{techniques}}
- Use [cf_xarray](https://cf-xarray.readthedocs.io/en/latest/) and [pyproj](https://pyproj4.github.io/pyproj/stable/) to access geographic metadata,
- Perform spatial subset of vector dataframe with [GeoPandas](https://geopandas.org/en/stable/),
- Clip raster data using vector data with [rioxarray](https://corteva.github.io/rioxarray/).
:::

{{break}}

Expand the next cell to see specific packages used in this notebook and relevant system and version information. 

In [ ]:
import cf_xarray
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pathlib
import pyproj

import itslive_tools

{{break}}

## A. Read data using strategy identified in previous notebook

Because we will be reading and writing files to disk, create a variable to hold the path to the root directory for this tutorial; we'll use this later on. 

In [ ]:
cwd = pathlib.Path.cwd()
tutorial1_dir = pathlib.Path(cwd).parent

In [ ]:
# Find url
url = itslive_tools.find_granule_by_point([95.180191, 30.645973])
# Read data cube without Dask
dc = itslive_tools.read_in_s3(url, chunks=None)
# Sort by mid-date
dc = dc.sortby("mid_date")
# Visually check mid-date in chronological order
dc.mid_date

Like in the last notebook, we want to chunk the dataset and to do that, we need to assign chunk sizes for each dimension. I copied the dictionary below from `dc_auto.chunksizes` in the previous notebook.

In [ ]:
chunking_dict = {
    "mid_date": (32736, 15156),
    "y": (
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        13,
    ),
    "x": (
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        13,
    ),
}

In [ ]:
dc = dc.chunk(chunking_dict)

In [ ]:
dc.chunks

Looks good 👍

## B. Incorporate glacier outline (vector) data

### {{b1_its_nb3}}

As discussed in the [Software and Data](software_and_data.ipynb#RGI) notebook, the examples in this tutorial use glacier outlines from the Randolph Glacier Inventory, version 7 (RGI7). We'll specifically be looking at the 'South Asia East' region.

In [ ]:
se_asia = gpd.read_parquet("../data/rgi7_region15_south_asia_east.parquet")

It is **vital** to check the CRS, or Coordinate Reference Systems, when combining geospatial data from different sources. 

The RGI data are in the `EPSG:4326` CRS.

In [ ]:
se_asia.crs

The CRS information for the ITS_LIVE dataset is stored in the `mapping` array. An easy way to discover this is to use the `cf_xarray` package and search for the `grid_mapping` variable if present.

In [ ]:
dc.cf

In [ ]:
cube_crs = pyproj.CRS.from_cf(dc.mapping.attrs)
cube_crs

This indicates that the data is projected to UTM zone 46N (EPSG:32646).

We choose to reproject the glacier outline to the CRS of the datacube:

In [ ]:
# Project rgi data to match itslive
se_asia_prj = se_asia.to_crs(cube_crs)
se_asia_prj.head(3)

How many glaciers are represented in the dataset?

In [ ]:
len(se_asia_prj)

### {{b2_its_nb3}}

In [Accessing S3 Data](accessing_s3_data.ipynb), we defined a function to create a vector object describing the footprint of a raster object; we'll use that again here.

In [ ]:
# Get vector bbox of itslive
bbox_dc = itslive_tools.get_bounds_polygon(dc)
bbox_dc["geometry"]
# Check that all objects have correct crs
assert dc.attrs["projection"] == bbox_dc.crs == se_asia_prj.crs

In [ ]:
# Plot the outline of the itslive granule and the rgi dataframe together
fig, ax = plt.subplots()

bbox_dc.plot(ax=ax, facecolor="None", color="red")
se_asia_prj.plot(ax=ax, facecolor="None")

### {{b3_its_nb3}}

The above plot shows the coverage of the vector dataset (`se_asia_prj`) in black, relative to the extent of the raster dataset (`bbox_dc`) in red. We use the [geopandas `.clip()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.clip.html) method to subset the RGI polygons  to the footprint of the ITS_LIVE datacube.

In [ ]:
# Subset rgi to bounds
se_asia_subset = gpd.clip(se_asia_prj, bbox_dc)
se_asia_subset.head()

We can use the `geopandas` `.explore()` method to interactively look at the RGI7 outlines contained within the ITS_LIVE granule:

In [ ]:
m = folium.Map(
    max_lat=31, max_lon=95, min_lat=29, min_lon=97, location=[30.2, 95.5], zoom_start=8
)

bbox_dc.explore(
    m=m,
    style_kwds={"fillColor": "None", "color": "red"},
    legend_kwds={"labels": ["ITS_LIVE granule footprint"]},
)
se_asia_subset.explore(m=m)

folium.LayerControl().add_to(m)
m

We can use the above interactive map to select a glacier to look at in more detail below.

However, notice that while the above code correctly produces a plot, it also throws a warning. We're going to ignore the warning for now, but if you're interested in a detailed example of how to trouble shoot and resolve this type of warning, check out the [appendix](../pt4/appendix.md).

We write `bbox_dc` to file so that we can use it in the appendix

In [ ]:
bbox_dc.to_file("../data/bbox_dc.geojson")

## C. Combine raster and vector data

### {{c1_its_nb3}}

If we want to dig in and analyze this velocity dataset at smaller spatial scales, we first need to subset it. The following section and the next notebook ([Exploratory data analysis of a single glacier](4_exploratory_data_analysis_single.ipynb)) will focus on the spatial scale of a single glacier. 

In [ ]:
# Select a  glacier to subset to
single_glacier_vec = se_asia_subset.loc[
    se_asia_subset["rgi_id"] == "RGI2000-v7.0-G-15-16257"
]
single_glacier_vec

In [ ]:
# Write it to file to that it can be used later
single_glacier_vec.to_file("../data/single_glacier_vec.json", driver="GeoJSON")

Check to see if the ITS_LIVE raster dataset has an assigned CRS attribute. We already know that the data is projected in the correct coordinate reference system (CRS), but the object may not be 'CRS-aware' yet (ie. have an attribute specifying its CRS). This is necessary for spatial operations such as clipping and reprojection. If `dc` doesn't have a CRS attribute, use `rio.write_crs()` to assign it. For more detail, see Rioxarray's [CRS Management documentation](https://corteva.github.io/rioxarray/stable/getting_started/crs_management.html).

In [ ]:
dc.rio.crs

Now, use the subset vector data object and Rioxarray's [`.clip()` method](https://corteva.github.io/rioxarray/html/examples/clip_geom.html) to crop the data cube.

In [ ]:
%%time

single_glacier_raster = dc.rio.clip(single_glacier_vec.geometry, single_glacier_vec.crs)

### {{c2_its_nb3}}
 
We want to use `single_glacier_raster` in the following notebook without going through all of the steps of creating it again. So, we write the object to file as a Zarr data cube so that we can easily read it into memory when we need it next. However, we'll see that there are a few steps we must go through before we can successfully write this object. 


We first re-chunk the `single_glacier_raster` into more optimal chunk sizes:

In [ ]:
single_glacier_raster = single_glacier_raster.chunk(
    {"mid_date": 20000, "x": 10, "y": 10}
)

If we try to write `single_glacier_raster`, eg: 

```single_glacier_raster.to_zarr('data/itslive/glacier_itslive.zarr', mode='w')```

We'll received an error related to encoding. 

The root cause is that the encoding recorded was appropriate for the source dataset, but is not valid anymore given all the transformations we have run up to this point. The easy solution here is to simply call `drop_encoding`. This will delete any existing encoding isntructions, and have Xarray automatically choose an encoding that will work well for the data. Optimizing the encoding of on-disk data is an advanced topic that we will not cover.

In [ ]:
single_glacier_raster = single_glacier_raster.persist()

In [ ]:
single_glacier_raster = single_glacier_raster.compute()

TODO
2/26 running this i got a ClientResponseError:500 when trying to write to zarr. Added in above two steps and it worked, not entirely sure why? if keep these in, add explanation.

In [ ]:
single_glacier_raster.drop_encoding().to_zarr(
    "../data/single_glacier_itslive.zarr", mode="w"
)

Now, let's try to write the object as a Zarr group.

{{conclusion}}

In this notebook, we read a large object into memory and clipped it to the footprint of a single area of using a large vector dataframe, all the while managing coordinate reference system metadata of both objects. We then saved the clipped raster object to disk so that it can be easily re-used. The next notebook demonstrates exploratory data analysis steps using the object we just wrote to disk. 